In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time
from draftstate import DraftState
import champion_info as cinfo
import match_processing as mp
from match_pool import test_train_split, match_pool

import qnetwork, softmax
from trainer import DDQNTrainer, SoftmaxTrainer
from inference_model import QNetInferenceModel, SoftmaxInferenceModel

import tensorflow as tf

In [2]:
print("********************************")
print("** Swain Bot 2.0 Start! **")
print("********************************")

valid_champ_ids = cinfo.get_ids()
print("Number of valid championIds: {}".format(len(valid_champ_ids)))

result = test_train_split()
validation_ids = result["validation_ids"]
training_ids = result["training_ids"]
print("Found {} training matches and {} validation matches in pool.".format(len(training_ids), len(validation_ids)))

validation_matches = match_pool(validation_ids)

print("***")
print("Displaying Validation matches:")
count = 0


for match in validation_matches['matches']:
    count += 1
    print("Match: {:2} id: {:4} winner: {:2}".format(count, match["id"], match["winner"]))
    for team in ["blue", "red"]:
        bans = match[team]["bans"]
        picks = match[team]["picks"]
        
        print("{} bans:{}".format(team, bans))
        print("{} picks:{}".format(team, picks))
    print("")
print("***")

********************************
** Swain Bot 2.0 Start! **
********************************
Number of valid championIds: 132
Found 810 training matches and 143 validation matches in pool.
***
Displaying Validation matches:
Match:  1 id:    0 winner:  0
blue bans:[52, 9, 115, 36, 120]
blue picks:[125, 93, 31, 122, 130]
red bans:[41, 73, 69, 70, 56]
red picks:[58, 1, 109, 35, 47]

Match:  2 id:    9 winner:  1
blue bans:[52, 106, 120, 91, 4]
blue picks:[41, 71, 1, 53, 64]
red bans:[125, 73, 9, 35, 115]
red picks:[101, 22, 116, 31, 5]

Match:  3 id:   21 winner:  0
blue bans:[87, 41, 73, 25, 31]
blue picks:[116, 100, 7, 71, 98]
red bans:[128, 101, 9, 85, 52]
red picks:[22, 53, 5, 70, 124]

Match:  4 id:   23 winner:  1
blue bans:[106, 41, 52, 93, 22]
blue picks:[120, 31, 101, 7, 79]
red bans:[1, 53, 125, 58, 24]
red picks:[73, 9, 69, 57, 62]

Match:  5 id:   31 winner:  1
blue bans:[27, 128, 98, 116, 85]
blue picks:[101, 73, 7, 124, 5]
red bans:[41, 9, 87, 79, 25]
red picks:[1, 100, 39, 

In [3]:
# Network parameters
state = DraftState(DraftState.BLUE_TEAM, valid_champ_ids)
input_size = state.format_state().shape
output_size = state.num_actions
filter_size = (1024,1024)
regularization_coeff = 7.5e-5#1.5e-4

# Training parameters
batch_size = 16#32
buffer_size = 4096#2048
n_epoch = 45
discount_factor = 0.9
learning_rate = 1.0e-4#2.0e-5#
time.sleep(2.)

for i in range(1):
    training_matches = match_pool(training_ids)
    print("Learning on {} matches for {} epochs. lr {:.4e} reg {:4e}".format(len(training_matches),n_epoch, learning_rate, regularization_coeff),flush=True)

    #tf.reset_default_graph()
    name = "softmax"
    out_path = "{}_model_E{}.ckpt".format(name, n_epoch)
    softnet = softmax.SoftmaxNetwork(name, out_path, input_size, output_size, filter_size, learning_rate, regularization_coeff)
    trainer = SoftmaxTrainer(softnet, n_epoch, training_matches, validation_matches, batch_size, load_path=None)
    summaries = trainer.train()

    #tf.reset_default_graph()
    name = "ddqn"
    out_path = "{}_model_E{}.ckpt".format(name, n_epoch)
    ddqn = qNetwork.Qnetwork(name, out_path, input_size, output_size, filter_size, learning_rate, regularization_coeff, discount_factor)
    trainer = DDQNTrainer(ddqn, n_epoch, training_matches, validation_matches, batch_size, buffer_size, load_path)
    summaries = trainer.train()

    print("Learning complete!")
    print("..final training accuracy: {:.4f}".format(summaries["train_acc"][-1]))
    x = [i+1 for i in range(len(summaries["loss"]))]
    fig = plt.figure()
    plt.plot(x,summaries["loss"])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    #plt.ylim([0,2])
    fig_name = "tmp/loss_figures/annuled_rate/loss_E{}_run_{}.pdf".format(n_epoch,i+1)
    print("Loss figure saved in:{}".format(fig_name),flush=True)
    fig.savefig(fig_name)

    fig = plt.figure()
    plt.plot(x, summaries["train_acc"], x, summaries["val_acc"])
    fig_name = "tmp/acc_figs/acc_E{}_run_{}.pdf".format(n_epoch,i+1)
    print("Accuracy figure saved in:{}".format(fig_name),flush=True)
    fig.savefig(fig_name)

Learning on 2 matches for 45 epochs. lr 1.0000e-04 reg 7.500000e-05
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\thebr\Desktop\492\softmax.py:84: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  fc0 = tf.compat.v1.layers.dense(
C:\Users\thebr\Desktop\492\softmax.py:92: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  fc1 = tf.compat.v1.layers.dense(
C:\Users\thebr\Desktop\492\softmax.py:101: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  ops_dict["logits"] = tf.compat.v1.layers.dense(


***
Beginning training..
  train_epochs: 45
  num_episodes: 2
  batch_size: 16
***
{'id': 655, 'winner': 0, 'blue': {'bans': [9, 18, 7, 98, 1], 'picks': [87, 86, 53, 25, 56]}, 'red': {'bans': [57, 128, 41, 28, 27], 'picks': [106, 24, 125, 31, 124]}}
memory (<draftstate.DraftState object at 0x0000024A2DE27670>, (7, -1), 0.95, <draftstate.DraftState object at 0x0000024A2DE276D0>)
memory (<draftstate.DraftState object at 0x0000024A2DDD4940>, (18, -1), 0.95, <draftstate.DraftState object at 0x0000024A2DE1CF70>)
memory (<draftstate.DraftState object at 0x0000024A2DE275E0>, (9, -1), 0.95, <draftstate.DraftState object at 0x0000024A2DE27460>)
memory (<draftstate.DraftState object at 0x0000024A2DE270A0>, (87, 87), -20.0, <draftstate.DraftState object at 0x0000024A2DE27970>)
memory (<draftstate.DraftState object at 0x0000024A2DE27E50>, (86, 86), -20.0, <draftstate.DraftState object at 0x0000024A2DE27E80>)
memory (<draftstate.DraftState object at 0x0000024A2DE27D90>, (53, 53), -20.0, <draftstate

TypeError: 'NoneType' object is not iterable

In [4]:
# Look at predicted Q values for states in a randomly drawn match

match = random.sample((list(training_matches['matches'])),1)

match = match[0]
match

{'id': 99,
 'winner': 1,
 'blue': {'bans': [31, 52, 1, 71, 74], 'picks': [9, 106, 28, 86, 100]},
 'red': {'bans': [120, 125, 101, 93, 22], 'picks': [73, 98, 128, 29, 58]}}

In [5]:
team = DraftState.RED_TEAM if match["winner"]==1 else DraftState.BLUE_TEAM
experiences = mp.process_match(match,team)
count = 0
# x labels for q val plots
xticks = []
xtick_locs = []
for a in range(state.num_actions):
    cid,pos = state.format_action(a)
    if cid not in xticks:
        xticks.append(cid)
        xtick_locs.append(a)
xtick_labels = [cinfo.champion_name_from_id(cid)[:6] for cid in xticks]


Winner 1 match_id 99
=== Begin Draft State ===
There are 5 picks and 10 bans completed in this draft. 

[52, 125, 31, 120, 1, 101, 22, 74, 93, 71]
[9, 106, 28, 86, 100]
Banned Champions: ['LeBlanc', 'Yuumi', 'Gwen', 'Wukong', 'Ahri', 'Taliyah', 'Draven', 'Pyke', 'Sivir', 'Ornn']
Picked Champions: ['Azir', 'Trundle', 'Gnar', 'Senna', 'Tahm Kench']
Enemy Draft: ['Azir', 'Gnar', 'Senna', 'Tahm Kench', 'Trundle']
Ally Draft:
Position 1: --
Position 2: --
Position 3: --
Position 4: --
Position 5: --
=== End Draft State ===
Error code 104
Number of experiences 9
(125, -1)
(120, -1)
(101, -1)
(73, 73)
(98, 98)
(128, 128)
(22, -1)
(93, -1)
(29, 29)

